# Single Run

In [1]:
import numpy as np
import pandas as pd
import pickle


from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master


from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary


from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


np.random.seed(seed = 0)


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_1401/3871698272.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Task: pazienti storici

In [2]:
historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [5]:
or_master_schedule = pd.read_csv("../not_git/master_schedule_input_v2.csv", sep = ';')
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [6]:
num_of_patients = 100

In [7]:
patients, training = patient_provider.provide_sets(quantity=num_of_patients, quantity_training=1000)


In [8]:
model = NGBLogNormal(description="test", patients=training)
patients = model.predict(patients)

In [9]:

task = Task(name = "My first problem",
            num_of_weeks= 3,
            num_of_patients= num_of_patients,
            robustness_risk= 0.2,
            robustness_overtime= 30,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients
task.master_schedule = master_schedule


In [10]:

task_file = open('task_100_1.pickle', 'ab')
     
# source, destination
pickle.dump(task, task_file)                    
task_file.close()


In [2]:
task_file = open('task.pickle', 'rb')
     
# source, destination
task = pickle.load(task_file)                    
task_file.close()

or_master_schedule = pd.read_csv("../not_git/master_schedule_input_v2.csv", sep = ';')

task.master_schedule = Master(name = "my master", table=or_master_schedule)

In [3]:
or_master_schedule

,weekday,equipes,room,duration
0,1,C,or1,420
1,1,H,or2,420
2,1,D,or3,420
3,1,L,or4,420
4,1,F,or5,420
5,2,I,or1,420
6,2,C,or2,420
7,2,F,or3,420
8,2,B,or4,420
9,2,A,or5,420


In [4]:
len(task.patients)

300

### Implementor - Adversary

In [11]:
task.set_robustness_overtime=20 #10
task.set_robustness_risk = 0.1

In [12]:
implementor_adversary = ImplementorAdversary(task = task, 
                                             implementor=StandardImplementor(), 
                                             adversary=None, 
                                             description = 'Test')

In [13]:
implementor_adversary.create_instance()

In [8]:
# implementor_adversary._implementor.instance_data

In [14]:
schedule, report = implementor_adversary.run(15)

implementor
adversary
2024-08-31 16:17:29.991957
2024-08-31 16:17:29.064704
0.015454216666666666


In [15]:
report

{'total duration': 0.015454216666666666,
 'start time': datetime.datetime(2024, 8, 31, 16, 17, 29, 64704),
 'end time': datetime.datetime(2024, 8, 31, 16, 17, 29, 991957),
 'number of iterations': 1,
 'is last robust': 'Yes',
 'iterations': [{'flag': True, 'duration': 0.015453933333333333}]}

In [16]:
schedule

{'task description': 'My first problem',
 'creation date': '2024-08-31 16:17:29',
 'number of blocks': 75,
 'blocks': [{'patients ids': [7731, 3142, 7237, 7712],
   'patients durations (nominal)': [85.4265263315217,
    86.13671142023458,
    86.30757175355059,
    85.49537314482178],
   'patients urgencies': [4, 4, 4, 4],
   'patients max waiting days': [360, 360, 360, 360],
   'patiens delay in days': [0, 0, 0, 0],
   'patient equipe': ['C', 'C', 'C', 'C'],
   'total duration (nominal)': 343.3661826501286,
   'block allocated duration': 420,
   'room': 'or1',
   'weekday': 1,
   'order_in_day': 1,
   'week': 0,
   'days since beginning': 0,
   'equipes': ['C'],
   'overtime probability': 0},
  {'patients ids': [870, 1758, 8600, 4861],
   'patients durations (nominal)': [83.36400063821607,
    83.36406089018452,
    112.7152275293268,
    82.5778414159121],
   'patients urgencies': [3, 1, 3, 4],
   'patients max waiting days': [180, 30, 180, 360],
   'patiens delay in days': [0, 0, 0,

### Chance constraint

In [17]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [18]:
chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [19]:
chance_constraints.create_instance()

In [20]:
#solution_chance_constraints = chance_constraints.run()
schedule, report = chance_constraints.run()

In [21]:
len(schedule.get('patients not included'))

29

In [22]:
schedule

{'task description': 'My first problem',
 'creation date': '2024-08-31 16:17:44',
 'number of blocks': 75,
 'blocks': [{'patients ids': [3142, 7237, 7712],
   'patients durations (nominal)': [86.13671142023458,
    86.30757175355059,
    85.49537314482178],
   'patients urgencies': [4, 4, 4],
   'patients max waiting days': [360, 360, 360],
   'patiens delay in days': [0, 0, 0],
   'patient equipe': ['C', 'C', 'C'],
   'total duration (nominal)': 257.9396563186069,
   'block allocated duration': 420,
   'room': 'or1',
   'weekday': 1,
   'order_in_day': 1,
   'week': 0,
   'days since beginning': 0,
   'equipes': ['C'],
   'overtime probability': 0},
  {'patients ids': [870, 1758, 8600, 4861],
   'patients durations (nominal)': [83.36400063821607,
    83.36406089018452,
    112.7152275293268,
    82.5778414159121],
   'patients urgencies': [3, 1, 3, 4],
   'patients max waiting days': [180, 30, 180, 360],
   'patiens delay in days': [0, 0, 0, 0],
   'patient equipe': ['H', 'H', 'H', 'H

In [23]:
report

{'total duration': 0.028796016666666667,
 'start time': datetime.datetime(2024, 8, 31, 16, 17, 43, 149145),
 'end time': datetime.datetime(2024, 8, 31, 16, 17, 44, 876906)}

### Budget Set

In [24]:
from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BudgetSetImplementor


In [25]:
task.gamma_variation = 0


budget_set = BudgetSet(task = task, 
                              implementor=BudgetSetImplementor(task = task), 
                              description = 'Test')






In [26]:
schedule, report = budget_set.run()

In [27]:
schedule

{'task description': 'My first problem',
 'creation date': '2024-08-31 16:18:01',
 'number of blocks': 75,
 'blocks': [{'patients ids': [7731, 3142, 7237, 7712],
   'patients durations (nominal)': [85.4265263315217,
    86.13671142023458,
    86.30757175355059,
    85.49537314482178],
   'patients urgencies': [4, 4, 4, 4],
   'patients max waiting days': [360, 360, 360, 360],
   'patiens delay in days': [0, 0, 0, 0],
   'patient equipe': ['C', 'C', 'C', 'C'],
   'total duration (nominal)': 343.3661826501286,
   'block allocated duration': 420,
   'room': 'or1',
   'weekday': 1,
   'order_in_day': 1,
   'week': 0,
   'days since beginning': 0,
   'equipes': ['C'],
   'overtime probability': 0},
  {'patients ids': [870, 1758, 8600, 4861],
   'patients durations (nominal)': [83.36400063821607,
    83.36406089018452,
    112.7152275293268,
    82.5778414159121],
   'patients urgencies': [3, 1, 3, 4],
   'patients max waiting days': [180, 30, 180, 360],
   'patiens delay in days': [0, 0, 0,

In [ ]:
budget_set._instance.get(None).get('time_increment')

In [ ]:
budget_set._instance.get(None).get('gamma')

In [ ]:
for block in soulution_budget_set._blocks:
    print(block)
    for pat in (block.patients):
        print(pat)
    print('---')